# Understanding Neural Networks at NeurIPS 2020

_Posted: 12/19/2020_

I attended NeurIPS 2020 this year.
The entire conference was online and, to be honest, I barely feel like I attended a conference at all.
I really just watched a lot of videos and bookmarked a lot of papers for future reading.
This is not an indictment of online conferences (I had an amazing time at EMNLP and ACL earlier this same year, socializing and really feeling present), but rather an admission that I had little energy or time to do more than keep on top of the content being presented.
However, in terms of the content (papers and presentations) of NeurIPS 2020 I had a pretty good experience.
First, the keynotes were excellent, and I left each one with new insights and inspiration. I would highly recommend watching them if you can find a link somewhere (I think you can find a public version [here](https://nips.cc/virtual/2020/public/f_inviteds.html).
Second, of course, there was a _ton_ of extremely interesting work, and being able to watch 3-minute videos over each paper felt like a very nice way to consume and comb through it.

The overall theme that excited me the most at NeurIPS 2020 was "understanding neural networks". As you may know, right now we are able to successfully use neural networks (NNs) in a broad range of machine learning tasks, from vision to speech to natural language.
However, despite all of their practical success, as a field we still have a relatively weak grasp on _just how_ neural networks are actually able to arrive at generalizable solutions, both theoretically and practically.
Moreover, modern NNs often exhibit pathological behaviors, such as being very susceptible to adversarial attacks or domain shift, and extrapoling nonsensical or stright-up harmful biases from data which make real-word uses dangerous.
The empirical success of these models, coupled with our lack of understanding of them and their behavior "out of a vaccuum", makes the "science of deep learning" a very exciting topic right now.
As such, most of my attention at this conference went to papers that attempt to explore or explain some component of deep neural network behavior, and my 10 favorite of those papers are what I put into this list.
I've ordered them by a grouping that I made completely up, which is:
- What do Neural Networks Learn?
- Leveraging Infinitely Wide Neural Networks
- The Lottery Ticket Hypothesis
- Bayesian Deep Learning

## What do Neural Networks Learn?

### 1. [The Pitfalls of Simplicity Bias in Neural Networks](https://proceedings.neurips.cc//paper_files/paper/2020/hash/6cfe0e6127fa25df2a0ef2ae1067d915-Abstract.html)

##### _Harshay Shah, Kaustav Tamuly, Aditi Raghunathan, Prateek Jain, Praneeth Netrapalli_

#### Quick Bits

Recent work has suggested that the **simplicity bias**, or the tendency of stochastic learning algorithms such as SGD to learn "simple" models, is able to explain why neural networks (NNs) are able to generalize despite their capacity to memorize the data. However, this begs the question - if NNs do indeed latch onto simple predictive features, why aren't they robust to adversarial attacks or distribution shift? One possible answer is that the simplicity bias actually causes this brittleness, by forcing an NN to latch onto idiosyncratic artifacts that are distribution specific.

In this work, the authors attempt to uncover the extent to which NNs are subject to the simplicity bias, and to what extent the simplicity bias _explains_ the brittleness of NNs in adversarial settings.
To probe this, the authors construct a set of datasets which contain predictive features of varying levels of simplicity (see figure below, from the paper).

![](blog_figs/neurips2020/simplicity_bias.png)

Training neural models on their synthetic datasets, the authors find:
- Neural networks are influenced by the simplicity bias in the extreme: models will latch onto only the simplest predictive feature, remaining completely invariant to all other features.
- The above can help explain NNs lack of robustness to small adversarial perturbations, and their reliance on spurious correlations which prevent OOD generalization.
- The simplicity bias also harms confidence estimations, as NNs derive confidences based solely on the simple predictive feature they have learned to rely on, even if other, more complex, predictive features contradict their confidence.
- Finally, NNs can actually learn to rely on simple features **even when those features are less predictive than more complex features**, suggesting that the simplicity bias can actually result in suboptimal models even for in-domain performance.

#### Why I like it

It's nice to see some evidence that the so-called simplicity bias induced by SGD is a double-edged sword. Yes, it gives us reasonable solutions, even when our models are vastly overparameterized and capable of interpolating the data... but it is also potentially the reason that NNs suffer from all kinds of issues with robustness. What's perhaps more interesting is that it suggests that there might be some optimal "simplicity" at which the model operates where we may still see good generalization and fewer problems with calibration, robustness, harmful biases etc. A less scalable solution is to try to create datasets which do not allow a model to rely on "too simple" a set of features to learn - perhaps a more scalable solution is to modify our learning algorithms to do so automatically.

### 2. [What Do Neural Networks Learn When Trained With Random Labels?](https://proceedings.neurips.cc//paper_files/paper/2020/hash/e4191d610537305de1d294adb121b513-Abstract.html)

##### _Hartmut Maennel, Ibrahim M. Alabdulmohsin, Ilya O. Tolstikhin, Robert Baldock, Olivier Bousquet, Sylvain Gelly, Daniel Keysers_

#### Quick Bits

We know that NNs trained on random labels can usually interpolate the data. This is often associated with memorization, which (to me) always meant nothing of the underlying distribution over _input and output_ space is learned. However, this does not preclude the possibility that an NN may still learn something about the distribution over _only_ the input domain.

In this paper, the authors first point out that, interestingly, training an NN on random labels results in a set of parameters that converge faster than randomly initialized parameters on true labels. This is a bit odd because it suggests that an NN trained on random lables, which is forced to memorize the dataset, is still _learning_ something that is useful to the correctly labeled task - i.e. the network is not "naively" memorizing the dataset, but instead is still trying to learn something generalizable during the memorization process.

Investigating this phenomenon further, the authors show:
- NNs trained on random labels do, in fact, learn something deep about the input space. They learn an alignment between the principal components of the network weights trained on random labels, and the principal components of the data.
- lower layers learn more generalizable features, and the upper layers are responsible for more 'memorization', or specializing on random labels, which can negatively impact transfer from random label weights to true labels.

#### Why I like it

I like this paper because, up to this point, I don't think I've ever really stopped to consider just _how_ an NN might go about actually memorizing a random dataset. I've always just taken it for granted that an NN trained on random labels will always be able to interpolate the data, but SGD will still find reasonable solutions when trained on ground-truth labels. My base assumption about interpolation was that it learns _nothing_ about the distribution it's trained on. So I find it fascinating that, even when in the setting of interpolation, NNs _do_ actually learn something about their input domain - SGD is still trying to learn generalizable features.


### 3. [What Neural Networks Memorize and Why: Discovering the Long Tail via Influence Estimation](https://proceedings.neurips.cc//paper_files/paper/2020/hash/1e14bfe2714193e7af5abc64ecbd6b46-Abstract.html)

##### _Vitaly Feldman, Chiyuan Zhang_

#### Quick Bits

Memorization is often associated with failures to successfully learn general rules in machine learning. In this paper, the authors argue that, when data distributions are long-tailed, memorization is actually a crucial part of learning.

The substantiate this claim, the authors:

1. Propose a novel way to measure the extent to which an example is memorized versus learned via general rules. Roughly speaking, this measurement is derived from the difference in model prediction when it has been trained on the example versus when it has not been trained on it.
2. Show that their proposed measurement is correlated with mislabeled or hard-to-label images.
3. Propose a very similar measurement to the one above to measure the influence (or importance) of a specific _training_ example on a specific _test_ example.
4. Show that some examples with high memorization scores are highly influential to exactly one or two images in the test set. Quantitatively, about 3% of ImageNet is composed of such pairs, and they examine this anecdotally in the figure below (from paper):
![](blog_figs/neurips2020/what_nns_memorize_1.png)



#### Why I like it 

I like this paper because it challenged my assumption that we don't want our models to memorize things - rather, we prefer if they learn general and robust rules. We may still _prefer_ if they learn general and robust rules, but it's interesting to consider that sometimes memorization may actually be optimal, particularly when we are thinking about performance on specific datasets. What's more, I think one of their claims is that some amount of memorization is _always_ optimal if you're dealing with long-tailed distributions, and thus this is true of any natural "real-world" dataset. I'm not sure if I'm completely on-board with this, but it's certainly a fun question to think about.

### 4. [Compositional Explanations of Neurons](https://proceedings.neurips.cc//paper_files/paper/2020/hash/c74956ffb38ba48ed6ce977af6727275-Abstract.html)

##### _Jesse Mu, Jacob Andreas_


#### Quick Bits

This work builds on NetDissect (Bau et al., 2017). The key motivation behind this work is that, while NetDissect offers a way to associate an individual neuron with a specific concept, in practice neurons are much more complex and can associate with complex abstractions which are _composed_ of more primitive concepts. To address this, the authors propose a way to construct logical compositions of concepts based on the framework of NetDissect (shown below, figure from paper).
![](blog_figs/neurips2020/compositional1.png)

Using this construction, the authors examine 3 key findings about neural networks:
- Neurons _do_ learn compositional concepts - they are able to show that explanation quality (as measured by IoU) increases as the length of logical compositions increases. Additionally, they find that neurons can be polysemantic, firing for multiple unrelated concepts.
- They find that interpretability improves accuracy - when neurons with high IoU scores fire the model is more likely to be correct.
- Finally, they find that they can use their compositions to predict how a model will behave when certain perturbations are made to the input (such as changing the water color).

#### Why I like it 

I think this paper is great because not only does it have very fun and interesting anecdotal results, but the quantitative results are pretty strong. I think it's cool that they're able to show that allowing for compositional explanations of neurons improves their interpretability, and that this interpretability is correlated with higher accuracy. On top of all of that, they do a very nice job of presenting their construction, which makes it feel very intuitive and simple. Interpretability of NNs is a very important subject at the moment, and this work feels like it makes some strong headways in this area. I would be really interested to see if we could use this IoU interpretability score as a pruning metric.

## Leveraging Infinitely Wide Neural Networks

### 5. [Deep learning versus kernel learning: an empirical study of loss landscape geometry and the time evolution of the Neural Tangent Kernel](https://proceedings.neurips.cc//paper_files/paper/2020/hash/405075699f065e43581f27d67bb68478-Abstract.html)

##### _Stanislav Fort, Gintare Karolina Dziugaite, Mansheej Paul, Sepideh Kharaghani, Daniel M. Roy, Surya Ganguli_


#### Quick Bits

This work attempts to unify recent findings in 3 different areas of deep learning research, which up until now have progressed relatively separately: (1) The global structure of deep learning loss landscapes, (2) the local geometry of deep learning loss landscapes, and (3) the neural tangent kernel (NTK).

To examine the relation between various phenomena in each area, the authors propose a set of experiments where they spawn children from parent trajectories during training, and examine various metrics of distance between these children as they continue training. These metrics each measure some property that is related to one of the subfields listed above: they include the error barrier (how high does the error get when linearly interpolating between the children), the distance between the NTKs of each child, how different are they in function space, how rapidly their individual NTKs change, and how well _linearized_ training of the child performs compared to standard, non-linear training.

With this comprehensive set of experiments, the authors empirically find that deep learning proceeds in 2 phases: an early, chaotic phase, followed by a more stable phase. Each phase shows correlations between phenomena that have been observed in prior work of each sub-field. Specifically, when spawning from parents during the early, chaotic phase:
- children arrive at diverse solutions, in function space
- children cannot be linearly interpolated without increasing loss
- linearized training cannot compete with full non-linear training
- children have NTKs which change significantly (i.e. are not constant)
- the distance between NTKs of different children is larger

However, quickly into training, parents enter the stable phase during which:
- children arrive at more similar solutions, and converge to the same basin of attraction
- children can be linearly interpolated without increasing loss
- linearized training can come very close to full non-linear training
- children have nearly constant NTKs
- the distance between NTKs of different children is small

The conclude by proposing the _non-linear_ advantage, which is the advantage that non-linear training has very early in training - they demonstrate that this advantage decreases rapidly as training progresses, until linear training is nearly as good as non-linear.

#### Why I like it 

This is the second paper on this list (the other being #7) which conduct a large scale study of deep learning in an attempt to improve our understanding of a specific part of deep learning. In this case, the goal of the authors is to demonstrate that various phenomena across sub-fields are related, and indeed can be unified by this perspective of the "chaotic and stable" training phases. I think work like this is important for a couple reasons:  first, large scale, carefully conducted empirical studies of NNs are currently our best way to really understand what's going on "under the hood", and they lead to very imporant discoveries (e.g. the lottery ticket hypothesis). Second, connecting various findings across different sub-fields to one, universal phenomena is a non-trivial task but results in a much more convincing finding since it is able to explain multiple results found in isolation from each other.

### 6. [On the linearity of large non-linear models: when and why the tangent kernel is constant](https://proceedings.neurips.cc//paper_files/paper/2020/hash/b7ae8fecf15b8b6c3c69eceae636d203-Abstract.html)

##### _Chaoyue Liu, Libin Zhu, Mikhail Belkin_


#### Quick Bits

Recent findings suggest that, as network width grows, some non-linear neural networks start to transition to linear functions with respect to their parameters, allowing their training dynamics to be linearly approximated. This is closely related to findings surrounding the NTK, which suggest that, for infinitely wide NNs with low learning rates, the NTK is constant.

In this work, the authors show that a constant NTK and linear dynamics are actually a property of the network being considered. Specifically, they identify two properties of a network that must hold as you increase the width of your network

1. The network must have a linear last layer.
2. The network cannot have a bottleneck layer.

They prove that if these 2 properties do not hold, then the network does not transition to linear training dynamics as the width increases, and the NTK is not constant. The general intuition for why this is true is that the transition to linearity depends on the norm of the Hessian - if both properties hold, then the norm of the Hessian goes to 0 as the width of the network increases. However, if they do not hold then the norm of the hessian does not go to zero as infinite width is approached, and thus the NTK is not constant (figure from presentation).
![](blog_figs/neurips2020/linear_ntk.png)

#### Why I like it

I'm really enjoying papers that deepen our understanding of the NTK. In this case, the paper introduces a couple failure modes where a constant NTK really fails to capture the training dynamics of infinite width networks, by very clearly explaining when and why the NTK is constant. I think this paper is easy to follow and presents an interesting and new piece of the puzzle to the NTK.

### 7. [Finite Versus Infinite Neural Networks: an Empirical Study](https://proceedings.neurips.cc//paper_files/paper/2020/hash/ad086f59924fffe0773f8d0ca22ea712-Abstract.html)

##### _Jaehoon Lee, Samuel Schoenholz, Jeffrey Pennington, Ben Adlam, Lechao Xiao, Roman Novak, Jascha Sohl-Dickstein_

#### Quick Bits

This paper attempts to address 2 conflicting claims in recent literature:

1. Increasing the size and depth of neural networks increases their performance and generalization capabilities.
2. Infinitely large neural networks, as characterized by NN gaussian processes or the NTK, often underperform standard finite neural networks.

To do this, the authors begin by noting that infinite neural networks are often unfairly compared to finite neural networks. For instance, finite NNs may use large learning rates or L2 regularization, which have no counterpart in infinite neural networks. To address this, the authors:
- identify sets of "interventions" that may confound comparisons between infinite and finite NNs.
- remove all interventions, resulting in a _base_ model that allows for fair comparison between infinite and finite NNs.
- iteratively add different "interventions" to each model to examine the effects on performance, and how that affects the relative performance of finite vs infinite NNs. Results are shown below (fig from paper). 
![](blog_figs/neurips2020/inf_vs_finite.png)

While the authors propose a number of findings that they uncover in this regime, the key findings are that:
- NTK and NNGPs _can_ outperform finite NNs in the base-model setting.
- Find that often NNGPs outperform NTK models.
- Large learning rates and L2 regularization drive the difference between the performance gap in finite NNs and infinite NNs.

#### Why I like it 

I like this paper because it feels very scientific. The authors have a clear goal in mind, which is to control confounding variables between infinite and finite NNs to allow for fair comparisons, and then to perform ablations to examine which variables are most important in practice. In doing so they uncover a number of findings with practical and theoretical implications. In particular, I like this paper because it's solely trying to improve our understanding of these models, and it's attempting to do so in a very principled way.

## The Lottery Ticket Hypothesis

### 8. [Optimal Lottery Tickets via Subset Sum: Logarithmic Over-Parameterization is Sufficient](https://proceedings.neurips.cc//paper_files/paper/2020/hash/1b742ae215adf18b75449c6e272fd92d-Abstract.html)

##### _Ankit Pensia, Shashank Rajput, Alliot Nagle, Harit Vishwakarma, Dimitris Papailiopoulos_


#### Quick Bits

The _strong lottery ticket hypothesis_  posits that, for any neural network with fixed weights, there exists a sufficiently overparameterized neural networks with random weights which can be pruned (without training) to approximate the smaller fixed network. However, there is some disparity between the latest bounds on the overparameterization, and what could be achieved in practice.

This paper provides a tighter bound on the overparameterization required to sustain the strong lottery ticket hypothesis. They do this with a clever construction that connects pruning randomly initialized networks to approximate a trained networks weights to random instances of the SubsetSum problem (pictured below, figure from paper). This problem has previously studied logarithmic bounds on requirements for set size, which this work leverages to get a tighter bound than previously known on overparameterization for the strong LTH.
![](blog_figs/neurips2020/lth.png)
Using this technique, the authors are able to present a bound on overparameterization which matches what has been accomplished in practice.

#### Why I like it 

The lottery ticket hypothesis, and subsequent strong lottery ticket hypothesis, are two of the most fascinating recent advances in understanding deep learning. I like this paper because it closes the gap between the theoretical bounds and empirical results surrounding the SLTH, which I think is important in helping us understand how we can better discover such subnetworks to begin with. Additionally, I love it when we can reduce one problem into another, more well-known problem, allowing us to leverage well-known results in understanding our current problem.

## Bayesian Deep Learning

### 9. [Liberty or Depth: Deep Bayesian Neural Nets Do Not Need Complex Weight Posterior Approximations](https://proceedings.neurips.cc//paper_files/paper/2020/hash/2dfe1946b3003933b7f8ddd71f24dbb1-Abstract.html)

##### _Sebastian Farquhar, Lewis Smith, Yarin Gal_

#### Quick /bits

In this work, the authors challenge the common belief that mean-field approximations are too restrictive for deep BNNs. They begin by noting 2 things:
- most prior work which suggests that the mean-field approximation is to restrictive only backs this claim with theory and empirical evidence on _small, shallow_ networks.
- rather than caring about the posterior distribution $p(\theta|D)$ directly, most applications of BNNs instead care about the predictive distribution $p(y | x) = \int_{\theta} p(y | \theta, x)p(\theta | D) \; d\theta$.

In this work, the authors combine these two observations two form 2 hypotheses about _deep_ mean-field BNNs:
- **Weight Distribution Hypothesis** - For any BNN with a full-covariance weight distribution, there
exists a deeper BNN with a mean-field weight distribution that induces a “similar” posterior
predictive distribution in function-space.
- **True Posterior Hypothesis** - For any sufficiently deep and wide BNN, and for any posterior predictive, there exists a mean-field distribution over the weights of that BNN which induces
the same distribution over function values as that induced by the posterior predictive, with
arbitrarily small error.

In the paper the authors provide analyic and empirical evidence that both of these hypotheses hold, suggesting that mean-field approximations for sufficiently deep BNNs are _not overly restrictive_, and can successfully be used to approximate the predictive posterior over functions.

#### Why I like it

This paper really struck me because it's something I've been thinking about recently.
Most of my knowledge of Bayesian Deep Learning is centered around MCMC-based sampling methods, and only very recently did I begin to ask msyelf why.
My immediate answer (to myself) was that it must just be way too difficult to come up with a factorizable distribution that can come close to approximating the true posterior of an NNs weights - after all, there must be so much dependence between weights, etc., and surely a mean-field approximation does not suffice.
So when I saw this paper, I was pretty excited to learn that I was wrong!

### 10. [Bayesian Deep Learning and a Probabilistic Perspective of Generalization](https://proceedings.neurips.cc//paper_files/paper/2020/hash/322f62469c5e3c7dc3e58f5a4d1ea399-Abstract.html)

##### _Andrew G. Wilson, Pavel Izmailov_


#### Quick Bits

Neural networks are known to interpolate randomly labeled data, and yet still generalize well when trained on true labels. In this work, the authors argue that, from a probabilistic perspective, this should not be surprising - namely, our models should have support for all possible datasets, including random labels, but should not _a priori_ assign much likelihood to such datasets (demonstrated below, figure from paper).

![](blog_figs/neurips2020/bayesian2.png)

To demonstrate this, the authors show that a Gaussian Process can also fit a dataset of random labels with perfect training accuracy - however, the GP assigns a low marginal likelihood to the dataset, suggesting that the noisy dataset has low support in the prior of the GP. They then show that the same is true of Bayesian NNs, suggesting that the ability of NNs to interpolate training data is not a flaw, but a natural desire for a hypothesis class with good support.

The authors then propose a novel bayesian deep learning method called Multi-SWAG, which extends the MCMC-based method SWAG to multiple separately trained trajectories, allowing Bayesian NNs to take advantage of ensembling over multiple basins of attraction.
While Multi-SWAG is able to outperform prior bayesian and ensemble methods on a number of tasks, 
perhaps it's most impressive result is that it is able to completely allieviate double descent (pictured below, figure from paper), which the authors argue should be expected when a proper probabilistic perspective is used.
![](blog_figs/neurips2020/bayesian3.png)


#### Why I like it 

First off, I think this paper is very well written and extremely clear. It also has some very convincing arguments for why we should be considering bayesian approaches to deep learning more seriously. However, my favorite part of this paper is that it changed my perspective on over-parameterization and interpolation - namely, that increasing model complexity when we have a good inductive bias already baked into our model _should_ increase our performance, and this is explained by the marginal likelihood of the data given by our model class. That they are able to provide a bayesian NN which alleviates double-descent all together only serves to makes their argument much more convincing.